# Committing and Rolling Back a Transaction


In [17]:
import mysql.connector as mysql
import os 
from dotenv import load_dotenv
import pandas as pd

In [2]:
!docker stop mysql-container
!docker restart mysql-container
!docker ps

mysql-container
mysql-container
CONTAINER ID   IMAGE     COMMAND                  CREATED      STATUS                  PORTS                                                  NAMES
50e33c579e3f   mysql     "docker-entrypoint.s…"   3 days ago   Up Less than a second   0.0.0.0:3306->3306/tcp, :::3306->3306/tcp, 33060/tcp   mysql-container


In [4]:
load_dotenv('/workspaces/IBM-DS-Course/.env')
user=os.getenv('USER')
password=os.getenv('PASSWORD')
host = 'localhost'
port=3306

conn = mysql.connect(
        host=host,
        user=user,
        password=password,
        port=port
        )

db = 'Transaction'

cursor = conn.cursor()
create_db =f'CREATE DATABASE IF NOT EXISTS {db}'
cursor.execute(create_db)
conn.commit()
cursor.close()
conn.close()


In [5]:
def get_db_connection():
    return mysql.connect(
        host=host, user=user, password=password,port=port, database=db

    )

In [11]:
conn = get_db_connection()
cursor = conn.cursor()
sql_scripts = ['/workspaces/IBM-DS-Course/Course 6 Db and SQL /Module 6 /1.3 .sql/BankAccounts-CREATE.sql', '/workspaces/IBM-DS-Course/Course 6 Db and SQL /Module 6 /1.3 .sql/ShoeShop-CREATE.sql']
for sql_script in sql_scripts:
    with open(sql_script, 'r') as script:
        script_cmd = script.read()

    use_db = f'USE {db}'
    cursor.execute(use_db)

    try:
        for cmd in script_cmd.split(";"):
            if cmd.strip():
                cursor.execute(cmd)
                print(f'executing {cmd}')
                rps = cursor.fetchall()
                print(*[row for row in rps], sep='\n')
            conn.commit()
            print("Success")
    except mysql.Error as e:
        print(f'{cmd} failed because e')
cursor.close()

executing 

DROP TABLE IF EXISTS BankAccounts

Success
executing 


CREATE TABLE BankAccounts (
    AccountNumber VARCHAR(5) NOT NULL,
    AccountName VARCHAR(25) NOT NULL,
    Balance DECIMAL(8,2) CHECK(Balance>=0) NOT NULL,
    PRIMARY KEY (AccountNumber)
    )

Success
executing 


    
INSERT INTO BankAccounts VALUES
('B001','Rose',300),
('B002','James',1345),
('B003','Shoe Shop',124200),
('B004','Corner Shop',76000)

Success
executing 


SELECT * FROM BankAccounts

('B001', 'Rose', Decimal('300.00'))
('B002', 'James', Decimal('1345.00'))
('B003', 'Shoe Shop', Decimal('124200.00'))
('B004', 'Corner Shop', Decimal('76000.00'))
Success
Success
executing DROP TABLE IF EXISTS ShoeShop

Success
executing 


CREATE TABLE ShoeShop (
    Product VARCHAR(25) NOT NULL,
    Stock INTEGER NOT NULL,
    Price DECIMAL(8,2) CHECK(Price>0) NOT NULL,
    PRIMARY KEY (Product)
    )

Success
executing 

INSERT INTO ShoeShop VALUES
('Boots',11,200),
('High heels',8,600),
('Brogues',10,150),
('Trainer

True

In [12]:
conn.close()

## Sample Exercise


1. Example of committing and rolling back a transaction.



- Scenario: Rose is buying a pair of boots from ShoeShop. So we have to update Rose's balance as well as the ShoeShop balance in the BankAccounts table. 
- Then we also have to update Boots stock in the ShoeShop table. 
- After Boots, let's also attempt to buy Rose a pair of Trainers.


2. Once the tables are ready, create a stored procedure routine named TRANSACTION_ROSE that includes TCL commands like COMMIT and ROLLBACK.


- Now develop the routine based on the given scenario to execute a transaction.
- To create the stored procedure routine on MySQL, copy the code below and paste it to the textarea of the SQL page. Click Go.

In [13]:
transaction = '''CREATE PROCEDURE TRANSACTION_ROSE()
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        RESIGNAL;
    END;
    START TRANSACTION;
    UPDATE BankAccounts
    SET Balance = Balance-200
    WHERE AccountName = 'Rose';
    UPDATE BankAccounts
    SET Balance = Balance+200
    WHERE AccountName = 'Shoe Shop';
    UPDATE ShoeShop
    SET Stock = Stock-1
    WHERE Product = 'Boots';
    UPDATE BankAccounts
    SET Balance = Balance-300
    WHERE AccountName = 'Rose';
    COMMIT;
END; '''

In [20]:
conn = get_db_connection()
cursor = conn.cursor()
check_exist = ''' SHOW PROCEDURE STATUS WHERE Name = 'TRANSACTION_ROSE';
''' # check if transc already exists
cursor.execute(check_exist)
check_exist_repsn = cursor.fetchall()
if not check_exist_repsn:
    cursor.execute(transaction)
    create_repsn = cursor.fetchall()
    reps_df = pd.DataFrame(create_repsn)
    reps_df
reps_df = pd.DataFrame(check_exist_repsn)
conn.commit()
cursor.close()
conn.close()

reps_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,Transaction,TRANSACTION_ROSE,PROCEDURE,SQL,root@%,2024-07-27 10:50:30,2024-07-27 10:50:30,DEFINER,,utf8mb4,utf8mb4_0900_ai_ci,utf8mb4_0900_ai_ci


3. Let's now check if the transaction can successfully be committed or not. Copy the code below in a new blank script and paste it to the textarea of the SQL page. Click Go


In [22]:
call1 = '''CALL TRANSACTION_ROSE;'''
select1 ='''SELECT * FROM BankAccounts;'''
select2 = '''SELECT * FROM ShoeShop;'''

In [24]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(call1)
re = cursor.fetchall()
repsn = cursor.fetchall()
print(*[row for row in repsn])
conn.commit()
cursor.close()
conn.close()

DatabaseError: 3819 (HY000): Check constraint 'BankAccounts_chk_1' is violated.

In [26]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(select1)
repsn = cursor.fetchall()
reps_df = pd.DataFrame(repsn)

conn.commit()
cursor.close()
conn.close()
reps_df

,0,1,2
0,B001,Rose,300.00
1,B002,James,1345.00
2,B003,Shoe Shop,124200.00
3,B004,Corner Shop,76000.00


In [27]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(select2)
repsn = cursor.fetchall()
reps_df = pd.DataFrame(repsn)

conn.commit()
cursor.close()
conn.close()
reps_df

,0,1,2
0,Boots,11,200.00
1,Brogues,10,150.00
2,High heels,8,600.00
3,Trainers,14,300.00


4. Observe that the transaction has been executed. But when we observe the tables, no changes have permanently been saved through COMMIT. 
    All the possible changes happened might have been undone through ROLLBACK since the whole transaction fails due to the failure of a SQL statement or more. 
    Let's go through the possible reason behind the failure of the transaction and how COMMIT - ROLLBACK works on a stored procedure:



- The first three UPDATEs should run successfully. 
- - Both the balance of Rose and ShoeShop should have been updated in the BankAccounts table. 
- - - The current balance of Rose should stand at 300 - 200 (price of a pair of Boots) = 100. 
- - - The current balance of ShoeShop should stand at 124,200 + 200 = 124,400. 
        
        The stock of Boots should also be updated in the ShoeShop table after the successful purchase for Rose, 11 - 1 = 10.

- The last UPDATE statement tries to buy Rose a pair of Trainers, but her balance becomes insufficient (Current balance of Rose: 100 < Price of Trainers: 300) after buying a pair of Boots. So, the last UPDATE statement fails. Since the whole transaction fails if any of the SQL statements fail, the transaction **won't be committed.**


## Practice exercise


Create a stored procedure TRANSACTION_JAMES to execute a transaction based on the following scenario: 
- First buy James 4 pairs of Trainers from ShoeShop:

- - Update his balance as well as the balance of ShoeShop. 
- - Also, update the stock of Trainers at ShoeShop. 

- Then attempt to buy James a pair of Brogues from ShoeShop. 
- - If any of the UPDATE statements fail, the whole transaction fails. 
- - You will roll back the transaction. 

- Commit the transaction only if the whole transaction is successful.

In [28]:
TRANSACTION_JAMES = '''CREATE PROCEDURE TRANSACTION_JAMES()
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        RESIGNAL;
    END;
    START TRANSACTION;
    UPDATE BankAccounts
    SET Balance = Balance-1200 
    WHERE AccountName = 'James';
    UPDATE BankAccounts
    SET Balance = Balance+1200
    WHERE AccountName = 'Shoe Shop';
    UPDATE ShoeShop
    SET Stock = Stock-4
    WHERE Product = 'Trainers';
    UPDATE BankAccounts
    SET Balance = Balance-150
    WHERE AccountName = 'James';
    COMMIT;
END '''

In [29]:
conn = get_db_connection()
cursor = conn.cursor()
check_exist = ''' SHOW PROCEDURE STATUS WHERE Name = 'TRANSACTION_JAMES';
''' # check if transc already exists
cursor.execute(check_exist)
check_exist_repsn = cursor.fetchall()
if not check_exist_repsn:
    cursor.execute(TRANSACTION_JAMES)
    create_repsn = cursor.fetchall()
    reps_df = pd.DataFrame(create_repsn)
    
reps_df = pd.DataFrame(check_exist_repsn)
conn.commit()
cursor.close()
conn.close()

reps_df

""


In [30]:
p_call1 = '''CALL TRANSACTION_JAMES;'''
p_select1 ='''SELECT * FROM BankAccounts;'''
p_select2 = '''SELECT * FROM ShoeShop;'''

In [31]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(p_call1)
re = cursor.fetchall()
repsn = cursor.fetchall()
print(*[row for row in repsn])
conn.commit()
cursor.close()
conn.close()

DatabaseError: 3819 (HY000): Check constraint 'BankAccounts_chk_1' is violated.

In [32]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(select1)
repsn = cursor.fetchall()
reps_df = pd.DataFrame(repsn)

conn.commit()
cursor.close()
conn.close()
reps_df

,0,1,2
0,B001,Rose,300.00
1,B002,James,1345.00
2,B003,Shoe Shop,124200.00
3,B004,Corner Shop,76000.00


In [33]:
conn = get_db_connection()
cursor = conn.cursor()
cursor.execute(select2)
repsn = cursor.fetchall()
reps_df = pd.DataFrame(repsn)

conn.commit()
cursor.close()
conn.close()
reps_df

,0,1,2
0,Boots,11,200.00
1,Brogues,10,150.00
2,High heels,8,600.00
3,Trainers,14,300.00


In [34]:
conn.close()